# AI e Aprendizado de Máquina no Databricks

##### Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro

Link de estudo:

* [docs databricks](https://docs.databricks.com/aws/pt/getting-started/quick-start)

## Consultar e visualizar dados no Databricks

In [1]:
# Só executar quando estiver trabalhando localmente:
import os
from dotenv import load_dotenv, find_dotenv


_ = load_dotenv(find_dotenv())

databricks_host = os.environ['DATABRICKS_HOST']
databricks_token = os.environ['DATABRICKS_TOKEN']
databricks_cluster_id = os.environ['DATABRICKS_CLUSTER_ID']

print("🔗 Carregado minhas credenciais do Databricks, com sucesso!")

🔗 Carregado minhas credenciais do Databricks, com sucesso!


In [2]:
from databricks.connect import DatabricksSession # Esta classe é usada para conectar ao Databricks remotamente
from pyspark.dbutils import DBUtils # Para usar dbutils localmente

spark = DatabricksSession.builder.remote(host=databricks_host, token=databricks_token, cluster_id=databricks_cluster_id).getOrCreate()

# Criar instância do DBUtils para usar dbutils localmente:
dbutils = DBUtils(spark)

print("✅ Conectado ao Databricks com sucesso!")
print(f"🔧 Versão Spark: {spark.version}")
print(f"📦 DBUtils configurado e pronto para uso!")

✅ Conectado ao Databricks com sucesso!
🔧 Versão Spark: 4.0.0
📦 DBUtils configurado e pronto para uso!


In [3]:
# Esta célula deve ser executada apenas quando estiver trabalhando localmente:

#databricks_cluster_id = spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

#print(f"Databricks Cluster ID: {databricks_cluster_id}")

In [4]:
# Consultando a seguinte Tabela "trips" que se encontra no 'samples' do Databricks:
taxi_df = spark.read.table("samples.nyctaxi.trips")

display(taxi_df) # Pode usar, também ---> display(taxi_df.limit(10))

DataFrame[tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, trip_distance: double, fare_amount: double, pickup_zip: int, dropoff_zip: int]

In [5]:
type(taxi_df)

pyspark.sql.connect.dataframe.DataFrame

In [6]:
# Exibir informações básicas da tabela
print("🚕 === DADOS DA TABELA NYC TAXI TRIPS ===")
print(f"📊 Total de registros (quantidade de dados/linhas): {taxi_df.count():,}")
print(f"📋 Número de colunas: {len(taxi_df.columns)}")
print(f"🏷️ O nomes das colunas são: {', '.join(taxi_df.columns)}")

print("\n📋 ESTRUTURA DA TABELA:")
taxi_df.printSchema()

🚕 === DADOS DA TABELA NYC TAXI TRIPS ===
📊 Total de registros (quantidade de dados/linhas): 21,932
📋 Número de colunas: 6
🏷️ O nomes das colunas são: tpep_pickup_datetime, tpep_dropoff_datetime, trip_distance, fare_amount, pickup_zip, dropoff_zip

📋 ESTRUTURA DA TABELA:
root
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_zip: integer (nullable = true)
 |-- dropoff_zip: integer (nullable = true)



In [7]:
# Usando o método show() para exibir: 
print("🔍 === PRIMEIRAS 7 LINHAS DA TABELA ===")
taxi_df.show(7, truncate=False) # Melhor usar "False" porque não vai curtar os textos longos.


🔍 === PRIMEIRAS 7 LINHAS DA TABELA ===
+--------------------+---------------------+-------------+-----------+----------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|trip_distance|fare_amount|pickup_zip|dropoff_zip|
+--------------------+---------------------+-------------+-----------+----------+-----------+
|2016-02-13 21:47:53 |2016-02-13 21:57:15  |1.4          |8.0        |10103     |10110      |
|2016-02-13 18:29:09 |2016-02-13 18:37:23  |1.31         |7.5        |10023     |10023      |
|2016-02-06 19:40:58 |2016-02-06 19:52:32  |1.8          |9.5        |10001     |10018      |
|2016-02-12 19:06:43 |2016-02-12 19:20:54  |2.3          |11.5       |10044     |10111      |
|2016-02-23 10:27:56 |2016-02-23 10:58:33  |2.6          |18.5       |10199     |10022      |
|2016-02-13 00:41:43 |2016-02-13 00:46:52  |1.4          |6.5        |10023     |10069      |
|2016-02-18 23:49:53 |2016-02-19 00:12:53  |10.4         |31.0       |11371     |10003      |
+--------------------

In [8]:
# Para saber o shape, fazemos no spark de diferente forma, assim:
num_linhas = taxi_df.count()
num_colunas = len(taxi_df.columns)

print(f"📊 O shape da tabela é: {num_linhas} linhas e {num_colunas} colunas")

📊 O shape da tabela é: 21932 linhas e 6 colunas


A seguir vou consultar meus dados `CSV` que tenho no meu `volume` dentro do `Catalog`:

In [9]:
df_volume = spark.read.csv("/Volumes/workspace/default_eddy/volumeeddy-tmp-sampledata/sample_data.csv", header=True, inferSchema=True)

print("Usando a função display:")
display(df_volume)

print("Usando o método show:")
df_volume.show(10, truncate=False)

Usando a função display:


DataFrame[Name: string, Age: int, City: string]

Usando o método show:
+-------+---+-----------+
|Name   |Age|City       |
+-------+---+-----------+
|Alice  |25 |New York   |
|Bob    |17 |Los Angeles|
|Charlie|35 |Chicago    |
|Diana  |16 |Houston    |
|Edward |45 |Phoenix    |
+-------+---+-----------+



## Importar e visualizar dados do CSV

Primeiro definimos, criamos o necessário para poder salvar nossos dados `"nome de bebês"` em nosso volume.

In [10]:
catalog = "catalogeddy"
schema = "catalogeddy_schema"
volume = "catalogeddy_volume"
download_url = "https://health.data.ny.gov/api/views/jxy9-yhdk/rows.csv" # São dados de nome de bebês
file_name = "baby_eddy.csv"
table_name = "tabel_baby_eddy"
path_volume = "/Volumes/" + catalog + "/" + schema + "/" + volume
path_table = catalog + "." + schema

print(f"O caminho completo da tabela é: {path_table}") 
print(f"O caminho completo do volume é:{path_volume}") 

O caminho completo da tabela é: catalogeddy.catalogeddy_schema
O caminho completo do volume é:/Volumes/catalogeddy/catalogeddy_schema/catalogeddy_volume


In [14]:
# Este comando copia os dados baixados e salva dentro do volume criado anteriormente:
#dbutils.fs.cp(f"{download_url}", f"{path_volume}" + "/" + f"{file_name}")

Agora que temos nossos dados salvos no volume `catalogeddy_volume` vamos a carregar esses dados `CSV` em um `DataFrame`:

In [15]:
df = spark.read.csv(f"{path_volume}/{file_name}",
  header=True,
  inferSchema=True,
  sep=",")

display(df)

DataFrame[Year: int, First Name: string, County: string, Sex: string, Count: int]

In [16]:
display(df)

DataFrame[Year: int, First Name: string, County: string, Sex: string, Count: int]

Agora, vamos salvar o `DataFrame` em uma tabela.

Na seguinte célula vamos substituir um espaço no nome da coluna. [Caracteres especiais](https://docs.databricks.com/aws/pt/sql/language-manual/sql-ref-names), como espaços, não são permitidos nos nomes das colunas.

In [17]:
df = df.withColumnRenamed("First Name", "First_Name")

df = df.withColumnRenamed("First_Name", "Primeiro_Nome") \
       .withColumnRenamed("Year", "Ano") \
           .withColumnRenamed("County", "Bairro") \
       .withColumnRenamed("Sex", "Sexo") \
       .withColumnRenamed("Count", "Contagem")

df.columns

['Ano', 'Primeiro_Nome', 'Bairro', 'Sexo', 'Contagem']

In [20]:
display(df.limit(3))

DataFrame[Ano: int, Primeiro_Nome: string, Bairro: string, Sexo: string, Contagem: int]

In [21]:
# Salvando o DataFrame como Tabela:
df.write.mode("overwrite").saveAsTable(f"{path_table}" + "." + f"{table_name}")